# Autor
Notebook criado por [Wallacy Pasqualini](https://www.linkedin.com/in/wallacypasqualini/).

Visite meu [GitHub](https://github.com/WallPasq) para ver meus projetos.

# O problema

> 💡 O objetivo desse Notebook está em mostrar o poder do Z-score. Leia até o final para entender o que foi feito em cada célula.

> ❗️ Abaixo conto uma história inspirada em algo que aconteceu comigo, com detalhes e exemplos fictícios para anonimizar devidamente os dados.

Recentemente passei por um desafio onde me foi passado uma meta de Ticket Médio geral, e eu precisava verificar o quanto o Ticket Médio de cada canal variou em relação à meta. 

Parece uma tarefa bem simples a princípio, certo? Só que tive dois problemas logo no início:

1. Os canais estavam segmentados em três colunas, onde cada coluna definia uma especificação diferente. Exemplo:

    | Tipo   | Canal      | Direcionamento   |
    |--------|------------|------------------|
    | Ads    | Facebook   | Blog             | 
    
    <br />
1. O Ticket Médio mudava muito dependendo do canal, o que deixava alguns muito acima da meta e outros muito abaixo. Exemplo:

    | Tipo   | Canal      | Direcionamento   | Meta     | Ticket Médio | Desvio |
    |--------|------------|------------------|----------|--------------|--------|
    | Ads    | Facebook   | Blog             | R$ 6.000 | R$ 10.000    | 67%    |
    | E-mail | Campanha   | Site             | R$ 6.000 | R$ 400       | - 93%  |

    <br />

O que você faria, nesse caso? Entregaria os números dessa forma?

Eu aprendi com o professor Daniel, na [Data Science Academy](https://www.datascienceacademy.com.br/), que o valor de um profissional na área de Dados está na sua capacidade de reconhecer e resolver problemas.

Por isso, para mim se tornou inaceitável entregar os valores dessa forma, pois estavam longe de representar a realidade.

Mas, como ajustar a meta de Ticket Médio para cada canal?

Talvez calcular um fator, que seria o Ticket Médio de cada combinação das três colunas dividido pelo Ticket Médio geral.

Depois bastaria multiplicar esse fator pela Meta, obtendo assim a Meta ajustada.

Essa seria uma possível solução, se não fosse o fato de eu precisar levar esses dados para o Power BI, permitindo filtros de seleção múltipla por cada coluna, que poderiam ser ou não combinados.

E agora? Talvez calcular o desvio no nível de cada coluna e também nos níveis das combinações?

Mas e se o número de colunas aumentasse depois? Certamente essa solução não seria viável nem escalável.

Isso além do fato da tabela ter milhões de linhas, o que prejudicaria a performance no Power BI para calcular o Ticket Médio de cada combinação e dividir pelo Ticket Médio geral, mesmo utilizando o contexto de filtro.

Não consegui encontrar uma solução naquele momento. Contudo, meu amigo [Gabriel Fonseca](https://www.linkedin.com/in/gabriel-felipe-fonseca/) me ensinou o seguinte:

> Às vezes é melhor dormir com a informação e deixar a nossa mente "pensar durante o repouso".

E foi o que eu fiz!

# Propriedades do Z-score

Você já ouviu falar no Z-score? Em uma definição simples, ele é a quantidade de desvios padrão que um valor está acima ou abaixo da média.

Por exemplo, em uma distribuição onde a média é 6.000 e o desvio padrão é 1.000, o valor 8.000 possui Z-score 2. Isso pois ele está 2 desvios padrão acima da média.

Já o valor 5.000 possui Z-score -1, pois ele está 1 desvio padrão abaixo da média.

Pegou a ideia? Sua fórmula é bem simples:

$$
\text{Z-score} = \frac{\text{Valor} - \text{Média}}{\text{Desvio padrão}}
$$

Dada sua fórmula, ele possui uma característica muito interessante: a soma de todos os Z-scores de um conjunto de dados é igual a 0.

> 💡 Isso se dá pela 1ª Propriedade da Média Aritmética, que diz que a soma dos desvios de cada valor em relação à média aritmética de um conjunto de dados é sempre igual a 0.

> 🧠 Não sabia disso? Bom, agora você sabe um dos motivos pelos quais elevamos os desvios ao quadrado para calcular o desvio padrão.

Se você já conhecia o Z-score e essa propriedade, um ponto que talvez você não sabia é que a média de um subconjunto de valores pode ser encontrada da seguinte forma:

$$
\text{Média do subconjunto} = \text{Média do conjunto} \cdot \text{Desvio padrão do conjunto} \cdot \text{Z-score médio do subconjunto}
$$

Onde o Z-score médio do subconjunto é igual a:

$$
\text{Z-score médio do subconjunto} = \frac{1}{\text{n}} \cdot \sum_{i=1}^{n}{\text{Z-score}_{i}}
\\~\\
\text{n} = \text{Quantidade de elementos do subconjunto}
$$

Essa propriedade é muito importante para a solução que apresentarei, e é demonstrada abaixo neste Notebook, no capítulo: [Encontrando a média de cada subconjunto com Z-score](#capitulo1).

> 💡 Note que estamos trazendo uma solução bem mais complexa e, ao mesmo tempo, mais plausível, do que simplesmente calcular o fator utilizando o Ticket Médio de cada canal.

# A solução

Se o Z-score médio pode ser utilizado para calcular a média de um subconjunto, ele também pode ser utilizado para ajustar a meta para cada subconjunto.

Contudo, calcular o Z-score médio para cada subconjunto cai na mesma problemática que levantei acima, principalmente de performance no Power BI.

Além, onde eu poderia calcular esse Z-score? Nos valores atuais? Isso não seria justo, pois a meta foi definida antes desses valores sequer serem gerados.

Assim, a solução encontrada foi levar alguns dados prontos para o Power BI, e realizar os demais cálculos lá, da seguinte forma:

1. Dados prontos:

    1. Calcular a soma dos Z-scores e a quantidade de elementos de cada combinação das três colunas através de dados históricos, por exemplo, do último mês:

        | Tipo   | Canal      | Direcionamento   | Soma Z-scores | Quantidade |
        |--------|------------|------------------|---------------|------------|
        | Ads    | Facebook   | Blog             | 100.000       | 75.000     |
        | E-mail | Campanha   | Site             | - 300.000     | 125.000    |

        <br />
    1. Calcular o Coeficiente de Variação dos dados históricos:

        $$
        \text{Coeficiente de Variação} = \frac{\text{Desvio padrão}}{\text{Média}}
        $$
    
    1. Calcular a variação da meta:

        $$
        \text{Variação da Meta} = \text{Meta} \cdot \text{Coeficiente de Variação}
        $$

1. Power BI:

    1. Ajustar a meta do Ticket Médio para cada canal:

        $$
        \text{Meta ajustada} = \text{Meta} + \text{Variação da Meta} \cdot \frac{\sum_{i=1}^{n}{\text{Soma Z-score}_{i}}}{\sum_{i=1}^{n}{\text{Quantidade}_{i}}}
        $$

        <br />

        Em DAX:

        ```language-dax
        meta_ajustada = DISTINCT(fMeta[meta]) + 
                        DISTINCT(fMeta[variacao]) *
                        DIVIDE(
                            SUM(fBaseHistorica[soma_z_score],
                            SUM(fBaseHistorica[quantidade]),
                            0))
        ```
        
        > 🧠 Note que a soma dos Z-scores dividido pela soma das quantidades nada mais é do que o Z-score médio do subconjunto.

Como ao invés da tabela inteira levamos uma tabela sumarizada para o Power BI, reduzimos drasticamente o número de linhas dela.

Abaixo, no exemplo deixado neste Notebook, você verá que uma sumarização parecida reduz uma tabela de 5 milhões de linhas para menos de 18 mil.

> 🎉 Uma redução de 99,6%!

Além disso, os cálculos deixados para o Power BI são executados rapidamente utilizando o contexto de filtro.

# Prática

Agora eu te convido a executar esse Notebook e acompanhar, através dos exemplos com dados aleatórios, o que foi feito e como a meta foi ajustada para cada canal.

Além, tente mudar os parâmetros da função base_aleatoria e analisar os resultados gerados.

Espero que você tenha aprendido algo comigo, e que possa levar esse aprendizado para seu futuro!

<a id='capitulo1'></a>
# Encontrando a média de cada subconjunto com Z-score

In [1]:
# Importa as bibliotecas
import pandas as pd
import numpy as np
import string
import time
from typing import List

In [2]:
# Define uma função para criar uma base aleatória com valores outliers
def base_aleatoria(tamanho: int = 5_000_000,
                   percentual_outliers_baixo: float = 0.05,
                   percentual_outliers_alto: float = 0.05,
                   classes_outliers_baixo: List[str] = [['Y1', 'Y2', 'Y3'],
                                                        ['Z1', 'Z2', 'Z3']],
                   classes_outliers_alto: List[str] = [['A1', 'A2', 'A3'],
                                                       ['B1', 'B2', 'B3'],
                                                       ['C1', 'C2', 'C3'],
                                                       ['D1', 'D2', 'D3']],
                   media_normal: float = 6_000,
                   desvio_padrao_normal: float = 500,
                   media_outlier_baixo: int = 2_000,
                   desvio_padrao_outlier_baixo: int = 100,
                   media_outlier_alto: int = 10_000,
                   desvio_padrao_outlier_alto: int = 2_000) -> pd.DataFrame:
    
    horario_inicio = time.time()
    letras = list(string.ascii_uppercase)
    
    quantidade_outliers_baixo = int(tamanho * percentual_outliers_baixo)
    quantidade_outliers_alto = int(tamanho * percentual_outliers_alto)
    tamanho_sem_outliers = tamanho - quantidade_outliers_baixo - quantidade_outliers_alto
    quantidade_outliers_baixo_por_classe = quantidade_outliers_baixo // len(classes_outliers_baixo)
    quantidade_outliers_alto_por_classe = quantidade_outliers_alto // len(classes_outliers_alto)

    print(f'Criando os primeiros {tamanho_sem_outliers:_.0f} valores'.replace('.', ',').replace('_', '.'))
    classe_1 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '1')
    classe_2 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '2')
    classe_3 = np.char.add(np.random.choice(letras, tamanho_sem_outliers), '3')
    valores = np.around(np.random.normal(media_normal, desvio_padrao_normal, tamanho_sem_outliers), 2)
    horario_fim_1 = time.time()
    print(f'Valores criados em {(horario_fim_1 - horario_inicio):_.2f} segundos'.replace('.', ',').replace('_', '.'))

    for i, cl in enumerate(classes_outliers_baixo, 1):
        print(f'\nCriando os {quantidade_outliers_baixo_por_classe:_.0f} outliers com valores baixos do {i}º conjunto de classes'.replace('.', ',').replace('_', '.'))
        classe_1 = np.concatenate((classe_1, np.repeat(cl[0], quantidade_outliers_baixo_por_classe)), axis=None)
        classe_2 = np.concatenate((classe_2, np.repeat(cl[1], quantidade_outliers_baixo_por_classe)), axis=None)
        classe_3 = np.concatenate((classe_3, np.repeat(cl[2], quantidade_outliers_baixo_por_classe)), axis=None)
        valores = np.concatenate((valores, np.around(np.random.normal(media_outlier_baixo, desvio_padrao_outlier_baixo, quantidade_outliers_baixo_por_classe), 2)), axis=None)
        horario_fim_2 = time.time()
        print(f'Valores criados em {(horario_fim_2 - horario_fim_1):_.2f} segundos'.replace('.', ',').replace('_', '.'))
        horario_fim_1 = horario_fim_2

    for i, cl in enumerate(classes_outliers_alto, 1):
        print(f'\nCriando os {quantidade_outliers_alto_por_classe:_.0f} outliers com valores altos do {i}º conjunto de classes'.replace('.', ',').replace('_', '.'))
        classe_1 = np.concatenate((classe_1, np.repeat(cl[0], quantidade_outliers_alto_por_classe)), axis=None)
        classe_2 = np.concatenate((classe_2, np.repeat(cl[1], quantidade_outliers_alto_por_classe)), axis=None)
        classe_3 = np.concatenate((classe_3, np.repeat(cl[2], quantidade_outliers_alto_por_classe)), axis=None)
        valores = np.concatenate((valores, np.around(np.random.normal(media_outlier_alto, desvio_padrao_outlier_alto, quantidade_outliers_alto_por_classe), 2)), axis=None)
        horario_fim_2 = time.time()
        print(f'Valores criados em {(horario_fim_2 - horario_fim_1):_.2f} segundos'.replace('.', ',').replace('_', '.'))
        horario_fim_1 = horario_fim_2

    base = pd.DataFrame(dict(classe_1=classe_1, classe_2=classe_2, classe_3=classe_3, valor=valores))
    print(f'\nBase criada em {(horario_fim_2 - horario_inicio):.2f} segundos'.replace('.', ',').replace('_', '.'))
    return base

In [3]:
# Gera a base histórica aleatória
base_historica = base_aleatoria()

# Obtém algumas estatísticas da base
quantidade_linhas_base_historica = len(base_historica)
media_base_historica = base_historica.valor.mean()
desvio_padrao_base_historica = base_historica.valor.std()
coeficiente_variacao_base_historica = desvio_padrao_base_historica / media_base_historica

# Calcula o Z-score de cada valor
base_historica['z_score'] = (base_historica.valor - media_base_historica) / desvio_padrao_base_historica
display(base_historica)

Criando os primeiros 4.500.000 valores
Valores criados em 9,86 segundos

Criando os 125.000 outliers com valores baixos do 1º conjunto de classes
Valores criados em 0,09 segundos

Criando os 125.000 outliers com valores baixos do 2º conjunto de classes
Valores criados em 0,09 segundos

Criando os 62.500 outliers com valores altos do 1º conjunto de classes
Valores criados em 0,09 segundos

Criando os 62.500 outliers com valores altos do 2º conjunto de classes
Valores criados em 0,10 segundos

Criando os 62.500 outliers com valores altos do 3º conjunto de classes
Valores criados em 0,10 segundos

Criando os 62.500 outliers com valores altos do 4º conjunto de classes
Valores criados em 0,10 segundos

Base criada em 10,44 segundos


,classe_1,classe_2,classe_3,valor,z_score
0,N1,G2,Z3,5571.58,-0.300916
1,L1,Z2,T3,6023.17,0.016501
2,J1,W2,O3,5685.65,-0.220738
3,V1,G2,S3,5715.51,-0.199750
4,O1,R2,R3,5246.84,-0.529173
...,...,...,...,...,...
4999995,D1,D2,D3,6753.07,0.529540
4999996,D1,D2,D3,10028.03,2.831474
4999997,D1,D2,D3,14941.11,6.284824
4999998,D1,D2,D3,6960.40,0.675270


In [4]:
# Mostra a quantidade de Linhas, a média geral do Valor e a soma do Z-score (a soma tem que ser zero)
print(f'Quantidade de Linhas: {quantidade_linhas_base_historica:_.0f}'.replace('.', ',').replace('_', '.'))
print(f'Média do Valor: {media_base_historica:_.2f}'.replace('.', ',').replace('_', '.'))
print(f'Soma do Z-score: {base_historica.z_score.sum():_.2f}'.replace('.', ',').replace('_', '.'))

Quantidade de Linhas: 5.000.000
Média do Valor: 5.999,69
Soma do Z-score: -0,00


In [5]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_1 = base_historica.groupby('classe_1').agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1.reset_index(inplace=True)
analise_classe_1['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1.soma_z_score / analise_classe_1.quantidade)
analise_classe_1.insert(3, 'proporcao', analise_classe_1.quantidade / quantidade_linhas_base_historica)
analise_classe_1.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_1.proporcao * analise_classe_1.media_real).sum()
resultado = 'ela é igual à média da base histórica' if np.isclose(media_ponderada, media_base_historica, atol=1e-08) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_1.soma_z_score.sum()
soma_quantidade = analise_classe_1.quantidade.sum()
soma_proporcao = analise_classe_1.proporcao.sum()
analise_classe_1 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1],
    ignore_index=True)

analise_classe_1['medias_iguais'] = np.isclose(analise_classe_1.media_real, analise_classe_1.media_calculada_com_z_score, atol=1e-08)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_1.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_1.proporcao = analise_classe_1.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_1.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 5.999,69 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"100,00%","5.999,69","5.999,69",True
1,B1,"175.803,67",235.576,"4,71%","7.061,41","7.061,41",True
2,D1,"175.375,20",235.213,"4,70%","7.060,46","7.060,46",True
3,A1,"175.297,16",235.320,"4,71%","7.059,51","7.059,51",True
4,C1,"175.494,97",236.008,"4,72%","7.057,61","7.057,61",True
5,M1,"256,00",173.167,"3,46%","6.001,80","6.001,80",True
6,P1,"220,91",173.348,"3,47%","6.001,51","6.001,51",True
7,H1,"214,58",173.068,"3,46%","6.001,46","6.001,46",True
8,G1,"192,55",173.670,"3,47%","6.001,27","6.001,27",True
9,J1,"188,90",173.353,"3,47%","6.001,24","6.001,24",True


In [6]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_2
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_2 = base_historica.groupby('classe_2').agg({'z_score': 'sum', 'classe_2': 'count', 'valor': 'mean'})
analise_classe_2.rename(columns={'z_score': 'soma_z_score', 'classe_2': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_2.reset_index(inplace=True)
analise_classe_2['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2.soma_z_score / analise_classe_2.quantidade)
analise_classe_2.insert(3, 'proporcao', analise_classe_2.quantidade / quantidade_linhas_base_historica)
analise_classe_2.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_2.proporcao * analise_classe_2.media_real).sum()
resultado = 'ela é igual à média da base histórica' if np.isclose(media_ponderada, media_base_historica, atol=1e-08) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_2.soma_z_score.sum()
soma_quantidade = analise_classe_2.quantidade.sum()
soma_proporcao = analise_classe_2.proporcao.sum()
analise_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_2],
    ignore_index=True)

analise_classe_2['medias_iguais'] = np.isclose(analise_classe_2.media_real, analise_classe_2.media_calculada_com_z_score, atol=1e-08)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_2.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_2.proporcao = analise_classe_2.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 5.999,69 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_2,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"100,00%","5.999,69","5.999,69",True
1,B2,"176.086,18",235.732,"4,71%","7.062,42","7.062,42",True
2,A2,"175.276,36",235.098,"4,70%","7.060,38","7.060,38",True
3,D2,"175.569,29",235.872,"4,72%","7.058,67","7.058,67",True
4,C2,"175.711,88",236.133,"4,72%","7.058,36","7.058,36",True
5,I2,"313,18",173.553,"3,47%","6.002,26","6.002,26",True
6,F2,"241,16",172.962,"3,46%","6.001,68","6.001,68",True
7,N2,"181,62",173.202,"3,46%","6.001,19","6.001,19",True
8,K2,"133,46",173.013,"3,46%","6.000,79","6.000,79",True
9,G2,"107,37",173.186,"3,46%","6.000,58","6.000,58",True


In [7]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_3
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_3 = base_historica.groupby('classe_3').agg({'z_score': 'sum', 'classe_3': 'count', 'valor': 'mean'})
analise_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_3': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_3.reset_index(inplace=True)
analise_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_3.soma_z_score / analise_classe_3.quantidade)
analise_classe_3.insert(3, 'proporcao', analise_classe_3.quantidade / quantidade_linhas_base_historica)
analise_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_3.proporcao * analise_classe_3.media_real).sum()
resultado = 'ela é igual à média da base histórica' if np.isclose(media_ponderada, media_base_historica, 0.1) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_3.quantidade.sum()
soma_proporcao = analise_classe_3.proporcao.sum()
analise_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_3=['Total'],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_3],
    ignore_index=True)

analise_classe_3['medias_iguais'] = np.isclose(analise_classe_3.media_real, analise_classe_3.media_calculada_com_z_score, atol=1e-08)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_3.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_3.proporcao = analise_classe_3.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 5.999,69 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,"-0,00",5.000.000,"100,00%","5.999,69","5.999,69",True
1,D3,"175.387,14",235.161,"4,70%","7.060,77","7.060,77",True
2,C3,"175.573,03",235.539,"4,71%","7.060,19","7.060,19",True
3,A3,"175.583,97",235.768,"4,72%","7.059,22","7.059,22",True
4,B3,"175.886,99",236.769,"4,74%","7.056,56","7.056,56",True
5,J3,"191,67",172.838,"3,46%","6.001,27","6.001,27",True
6,O3,"182,60",173.047,"3,46%","6.001,19","6.001,19",True
7,W3,"175,79",172.748,"3,45%","6.001,14","6.001,14",True
8,I3,"140,64",172.704,"3,45%","6.000,85","6.000,85",True
9,X3,"128,04",172.308,"3,45%","6.000,75","6.000,75",True


In [8]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1 + classe_2
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_1_classe_2 = base_historica.groupby(['classe_1', 'classe_2']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1_classe_2.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1_classe_2.reset_index(inplace=True)
analise_classe_1_classe_2['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade)
analise_classe_1_classe_2.insert(3, 'proporcao', analise_classe_1_classe_2.quantidade / quantidade_linhas_base_historica)
analise_classe_1_classe_2.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_1_classe_2.proporcao * analise_classe_1_classe_2.media_real).sum()
resultado = 'ela é igual à média da base histórica' if np.isclose(media_ponderada, media_base_historica, atol=1e-08) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_1_classe_2.soma_z_score.sum()
soma_quantidade = analise_classe_1_classe_2.quantidade.sum()
soma_proporcao = analise_classe_1_classe_2.proporcao.sum()
analise_classe_1_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1_classe_2],
    ignore_index=True)

analise_classe_1_classe_2['medias_iguais'] = np.isclose(analise_classe_1_classe_2.media_real, analise_classe_1_classe_2.media_calculada_com_z_score, atol=1e-08)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_1_classe_2.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_1_classe_2.proporcao = analise_classe_1_classe_2.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_1_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 5.999,69 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,classe_2,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"100,00%","5.999,69","5.999,69",True
1,B1,B2,"175.822,49",69.190,"1,38%","9.614,99","9.614,99",True
2,A1,A2,"175.519,04",69.098,"1,38%","9.613,56","9.613,56",True
3,C1,C2,"175.703,78",69.226,"1,38%","9.610,67","9.610,67",True
4,D1,D2,"175.447,00",69.251,"1,39%","9.604,09","9.604,09",True
...,...,...,...,...,...,...,...,...
672,C1,N2,"-64,74",6.597,"0,13%","5.985,73","5.985,73",True
673,K1,R2,"-68,08",6.674,"0,13%","5.985,18","5.985,18",True
674,T1,Z2,"-69,28",6.692,"0,13%","5.984,96","5.984,96",True
675,Z1,Z2,"-351.372,83",131.723,"2,63%","2.204,62","2.204,62",True


In [9]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_1 + classe_3
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_1_classe_3 = base_historica.groupby(['classe_1', 'classe_3']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_classe_1_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_1_classe_3.reset_index(inplace=True)
analise_classe_1_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade)
analise_classe_1_classe_3.insert(3, 'proporcao', analise_classe_1_classe_3.quantidade / quantidade_linhas_base_historica)
analise_classe_1_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_1_classe_3.proporcao * analise_classe_1_classe_3.media_real).sum()
resultado = 'ela é igual à média da base histórica' if np.isclose(media_ponderada, media_base_historica, atol=1e-08) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_1_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_1_classe_3.quantidade.sum()
soma_proporcao = analise_classe_1_classe_3.proporcao.sum()
analise_classe_1_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_1_classe_3],
    ignore_index=True)

analise_classe_1_classe_3['medias_iguais'] = np.isclose(analise_classe_1_classe_3.media_real, analise_classe_1_classe_3.media_calculada_com_z_score, atol=1e-08)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_1_classe_3.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_1_classe_3.proporcao = analise_classe_1_classe_3.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_1_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 5.999,69 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"100,00%","5.999,69","5.999,69",True
1,B1,B3,"175.814,36",69.186,"1,38%","9.615,03","9.615,03",True
2,D1,D3,"175.477,44",69.135,"1,38%","9.610,77","9.610,77",True
3,C1,C3,"175.649,12",69.229,"1,38%","9.609,39","9.609,39",True
4,A1,A3,"175.461,51",69.276,"1,39%","9.603,09","9.603,09",True
...,...,...,...,...,...,...,...,...
672,A1,T3,"-85,77",6.501,"0,13%","5.980,92","5.980,92",True
673,E1,M3,"-86,10",6.507,"0,13%","5.980,87","5.980,87",True
674,N1,V3,"-102,50",6.591,"0,13%","5.977,57","5.977,57",True
675,Z1,Z3,"-351.380,97",131.700,"2,63%","2.203,87","2.203,87",True


In [10]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento da classe_2 + classe_3
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_classe_2_classe_3 = base_historica.groupby(['classe_2', 'classe_3']).agg({'z_score': 'sum', 'classe_2': 'count', 'valor': 'mean'})
analise_classe_2_classe_3.rename(columns={'z_score': 'soma_z_score', 'classe_2': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_classe_2_classe_3.reset_index(inplace=True)
analise_classe_2_classe_3['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade)
analise_classe_2_classe_3.insert(3, 'proporcao', analise_classe_2_classe_3.quantidade / quantidade_linhas_base_historica)
analise_classe_2_classe_3.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_classe_2_classe_3.proporcao * analise_classe_2_classe_3.media_real).sum()
resultado = 'ela é igual à média da base histórica' if np.isclose(media_ponderada, media_base_historica, atol=1e-08) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_classe_2_classe_3.soma_z_score.sum()
soma_quantidade = analise_classe_2_classe_3.quantidade.sum()
soma_proporcao = analise_classe_2_classe_3.proporcao.sum()
analise_classe_2_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_classe_2_classe_3],
    ignore_index=True)

analise_classe_2_classe_3['medias_iguais'] = np.isclose(analise_classe_2_classe_3.media_real, analise_classe_2_classe_3.media_calculada_com_z_score, atol=1e-08)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_classe_2_classe_3.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_classe_2_classe_3.proporcao = analise_classe_2_classe_3.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_classe_2_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 5.999,69 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_2,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,"-0,00",5.000.000,"100,00%","5.999,69","5.999,69",True
1,B2,B3,"175.881,45",69.128,"1,38%","9.619,45","9.619,45",True
2,C2,C3,"175.658,04",69.127,"1,38%","9.614,90","9.614,90",True
3,D2,D3,"175.470,38",69.072,"1,38%","9.613,92","9.613,92",True
4,A2,A3,"175.525,57",69.164,"1,38%","9.610,24","9.610,24",True
...,...,...,...,...,...,...,...,...
672,J2,G3,"-78,77",6.706,"0,13%","5.982,98","5.982,98",True
673,F2,L3,"-79,22",6.679,"0,13%","5.982,82","5.982,82",True
674,V2,G3,"-83,23",6.675,"0,13%","5.981,95","5.981,95",True
675,Y2,Y3,"-351.474,69",131.662,"2,63%","2.201,76","2.201,76",True


In [11]:
# Calcula a soma do Z-score, a quantidade de registros, a média real do Valor e a média calculada com o Z-score para cada elemento de todas as classes
# Calcula também a proporção da quantidade de boletos e mostra que a media ponderada pela proporção é igual à media histórica
analise_geral = base_historica.groupby(['classe_1', 'classe_2', 'classe_3']).agg({'z_score': 'sum', 'classe_1': 'count', 'valor': 'mean'})
analise_geral.rename(columns={'z_score': 'soma_z_score', 'classe_1': 'quantidade', 'valor': 'media_real'}, inplace=True)
analise_geral.reset_index(inplace=True)
analise_geral['media_calculada_com_z_score'] = media_base_historica + desvio_padrao_base_historica * (analise_geral.soma_z_score / analise_geral.quantidade)
analise_geral.insert(3, 'proporcao', analise_geral.quantidade / quantidade_linhas_base_historica)
analise_geral.sort_values(by='media_calculada_com_z_score', ascending=False, ignore_index=True, inplace=True)

media_ponderada = (analise_geral.proporcao * analise_geral.media_real).sum()
resultado = 'ela é igual à média da base histórica' if np.isclose(media_ponderada, media_base_historica, atol=1e-08) else \
            'ela é diferente da média da base histórica'
print(f'A média ponderada é {media_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_z_score = analise_geral.soma_z_score.sum()
soma_quantidade = analise_geral.quantidade.sum()
soma_proporcao = analise_geral.proporcao.sum()
analise_geral = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      classe_3=[''],
                      soma_z_score=[soma_z_score],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      media_real=[media_base_historica],
                      media_calculada_com_z_score=[media_base_historica + desvio_padrao_base_historica * (soma_z_score / soma_quantidade)])),
    analise_geral],
    ignore_index=True)

analise_geral['medias_iguais'] = np.isclose(analise_geral.media_real, analise_geral.media_calculada_com_z_score, atol=1e-08)
print('Todas as médias reais são iguais às médias calculadas com z_score.' if analise_geral.medias_iguais.all() else \
      'Ao menos uma das médias reais é diferente da média calculada com z_score.')

analise_geral.proporcao = analise_geral.proporcao.apply(lambda x: f'{x:.2%}'.replace('.', ','))
display(analise_geral.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A média ponderada é 5.999,69 e ela é igual à média da base histórica.
Todas as médias reais são iguais às médias calculadas com z_score.


,classe_1,classe_2,classe_3,soma_z_score,quantidade,proporcao,media_real,media_calculada_com_z_score,medias_iguais
0,Total,,,"-0,00",5.000.000,"100,00%","5.999,69","5.999,69",True
1,B1,B2,B3,"175.824,53",62.759,"1,26%","9.985,50","9.985,50",True
2,C1,C2,C3,"175.688,58",62.786,"1,26%","9.980,71","9.980,71",True
3,A1,A2,A3,"175.492,39",62.772,"1,26%","9.977,15","9.977,15",True
4,D1,D2,D3,"175.440,76",62.764,"1,26%","9.976,49","9.976,49",True
...,...,...,...,...,...,...,...,...,...
17572,B1,Q2,M3,"-20,04",250,"0,01%","5.885,65","5.885,65",True
17573,G1,B2,E3,"-23,32",282,"0,01%","5.882,06","5.882,06",True
17574,O1,V2,K3,"-22,70",246,"0,00%","5.868,39","5.868,39",True
17575,Z1,Z2,Z3,"-351.387,74",125.243,"2,50%","2.008,10","2.008,10",True


<a id='capitulo2'></a>
# Reajustando a meta com uma nova base

In [12]:
# Cria uma nova base para reajustar a meta
base = base_aleatoria(tamanho=1_000_000,
                      media_normal=7_000)

# Define um valor para a meta e calcula a variação da meta
meta = 7_000
variacao_meta = meta * coeficiente_variacao_base_historica

# Imprime a base na tela
display(base)

Criando os primeiros 900.000 valores
Valores criados em 1,83 segundos

Criando os 25.000 outliers com valores baixos do 1º conjunto de classes
Valores criados em 0,02 segundos

Criando os 25.000 outliers com valores baixos do 2º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 1º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 2º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 3º conjunto de classes
Valores criados em 0,02 segundos

Criando os 12.500 outliers com valores altos do 4º conjunto de classes
Valores criados em 0,02 segundos

Base criada em 1,94 segundos


,classe_1,classe_2,classe_3,valor
0,U1,Y2,B3,7193.67
1,G1,C2,M3,6761.24
2,C1,K2,O3,6267.54
3,R1,N2,F3,6523.51
4,D1,R2,A3,7404.82
...,...,...,...,...
999995,D1,D2,D3,7856.02
999996,D1,D2,D3,10032.97
999997,D1,D2,D3,11364.68
999998,D1,D2,D3,13294.72


In [13]:
# Calcula a média geral do Valor
media_base = base.valor.mean()
print(f'Média do Valor: {media_base:_.2f}'.replace('.', ',').replace('_', '.'))

Média do Valor: 6.900,49


In [14]:
# Resume a base e a base histórica para juntá-las, visando facilitar os cálculos
base = base.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).valor.agg(valor='sum', quantidade='count')
base['proporcao'] = base.quantidade / base.quantidade.sum()
base_historica_resumida = base_historica.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).z_score.agg(soma_z_score='sum', quantidade_historica='count')
base_historica_resumida['proporcao_historica'] = base_historica_resumida.quantidade_historica / base_historica_resumida.quantidade_historica.sum()
base = base.merge(base_historica_resumida, how='outer', on=['classe_1', 'classe_2', 'classe_3']).fillna(0)
base = base.astype({'valor': float, 'quantidade': int, 'soma_z_score': float, 'quantidade_historica': int, 'proporcao_historica': float})
display(base)

,classe_1,classe_2,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica
0,A1,A2,A3,1.256547e+08,12553,0.012553,175492.389136,62772,0.012554
1,A1,A2,B3,3.673989e+05,53,0.000053,2.704392,261,0.000052
2,A1,A2,C3,3.946867e+05,56,0.000056,-5.012182,246,0.000049
3,A1,A2,D3,3.065688e+05,44,0.000044,0.477689,266,0.000053
4,A1,A2,E3,3.070627e+05,44,0.000044,-3.397516,253,0.000051
...,...,...,...,...,...,...,...,...,...
17571,Z1,Z2,V3,3.667391e+05,52,0.000052,7.342037,240,0.000048
17572,Z1,Z2,W3,3.677105e+05,53,0.000053,8.494275,257,0.000051
17573,Z1,Z2,X3,3.310659e+05,47,0.000047,9.341037,263,0.000053
17574,Z1,Z2,Y3,3.962535e+05,57,0.000057,-0.291158,241,0.000048


In [15]:
# Calcula os desvios da classe_1 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1 = base.groupby('classe_1', as_index=False).sum(numeric_only=True)
analise_classe_1['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1.soma_z_score / analise_classe_1.quantidade_historica)
analise_classe_1['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1.soma_z_score / analise_classe_1.quantidade_historica)
analise_classe_1['valor_medio'] = base.valor / base.quantidade
analise_classe_1['desvio_valor'] = (analise_classe_1.valor_medio / analise_classe_1.meta_ajustada_com_z_score) - 1
analise_classe_1['desvio_proporcao'] = (analise_classe_1.proporcao / analise_classe_1.proporcao_historica) - 1
analise_classe_1.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_1.proporcao_historica * analise_classe_1.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if np.isclose(meta_ponderada, float(meta), atol=1e-08) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_1.valor.sum()
soma_quantidade = analise_classe_1.quantidade.sum()
soma_proporcao = analise_classe_1.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_1.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_1 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_1],
    ignore_index=True)

analise_classe_1.proporcao = analise_classe_1.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1.proporcao_historica = analise_classe_1.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1.desvio_valor = analise_classe_1.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1.desvio_proporcao = analise_classe_1.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,"6.900.492.796,26",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","5.999,69","7.000,00","6.900,49","-1,42%","0,00%"
1,Z1,"294.425.257,28",59.919,"5,99%","-351.318,29",298.641,"5,97%","4.326,04","5.047,31","6.983,77","38,37%","0,32%"
2,Y1,"292.459.111,41",59.656,"5,97%","-351.410,18",297.907,"5,96%","4.321,48","5.041,99","6.882,06","36,50%","0,13%"
3,A1,"367.573.780,17",47.097,"4,71%","175.297,16",235.320,"4,71%","7.059,51","8.236,51","10.009,93","21,53%","0,07%"
4,N1,"242.876.872,95",34.686,"3,47%","-299,60",173.416,"3,47%","5.997,24","6.997,13","7.060,94","0,91%","0,01%"
5,R1,"240.929.502,97",34.403,"3,44%","-95,74",172.481,"3,45%","5.998,90","6.999,08","7.050,34","0,73%","-0,27%"
6,O1,"242.888.141,27",34.701,"3,47%","-121,98",173.392,"3,47%","5.998,69","6.998,83","7.043,96","0,64%","0,07%"
7,J1,"240.413.570,62",34.361,"3,44%","188,90",173.353,"3,47%","6.001,24","7.001,81","7.033,58","0,45%","-0,89%"
8,H1,"242.466.758,22",34.646,"3,46%","214,58",173.068,"3,46%","6.001,46","7.002,06","7.030,39","0,40%","0,09%"
9,M1,"241.934.021,61",34.581,"3,46%","256,00",173.167,"3,46%","6.001,80","7.002,45","7.024,73","0,32%","-0,15%"


In [16]:
# Calcula os desvios da classe_2 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_2 = base.groupby('classe_2', as_index=False).sum(numeric_only=True)
analise_classe_2['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2.soma_z_score / analise_classe_2.quantidade_historica)
analise_classe_2['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_2.soma_z_score / analise_classe_2.quantidade_historica)
analise_classe_2['valor_medio'] = base.valor / base.quantidade
analise_classe_2['desvio_valor'] = (analise_classe_2.valor_medio / analise_classe_2.meta_ajustada_com_z_score) - 1
analise_classe_2['desvio_proporcao'] = (analise_classe_2.proporcao / analise_classe_2.proporcao_historica) - 1
analise_classe_2.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_2.proporcao_historica * analise_classe_2.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if np.isclose(meta_ponderada, float(meta), atol=1e-08) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_2.valor.sum()
soma_quantidade = analise_classe_2.quantidade.sum()
soma_proporcao = analise_classe_2.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_2.soma_z_score.sum()
soma_quantidade_historica = analise_classe_2.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_2.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_2],
    ignore_index=True)

analise_classe_2.proporcao = analise_classe_2.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2.proporcao_historica = analise_classe_2.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2.desvio_valor = analise_classe_2.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2.desvio_proporcao = analise_classe_2.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_2,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,"6.900.492.796,26",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","5.999,69","7.000,00","6.900,49","-1,42%","0,00%"
1,Z2,"293.591.529,71",59.776,"5,98%","-351.578,38",298.118,"5,96%","4.321,87","5.042,44","6.983,77","38,50%","0,26%"
2,Y2,"289.645.430,91",59.230,"5,92%","-351.411,30",297.673,"5,95%","4.320,16","5.040,44","6.882,06","36,54%","-0,51%"
3,A2,"365.915.934,32",46.908,"4,69%","175.276,36",235.098,"4,70%","7.060,38","8.237,53","10.009,93","21,52%","-0,24%"
4,N2,"241.776.283,64",34.550,"3,45%","181,62",173.202,"3,46%","6.001,19","7.001,74","7.060,94","0,85%","-0,26%"
5,R2,"242.058.478,37",34.603,"3,46%","-224,83",172.928,"3,46%","5.997,84","6.997,84","7.050,34","0,75%","0,05%"
6,O2,"240.328.602,80",34.338,"3,43%","-65,38",173.315,"3,47%","5.999,16","6.999,37","7.043,96","0,64%","-0,94%"
7,J2,"241.412.634,94",34.486,"3,45%","-48,60",173.378,"3,47%","5.999,29","6.999,53","7.033,58","0,49%","-0,55%"
8,H2,"241.337.481,26",34.462,"3,45%","63,86",173.089,"3,46%","6.000,22","7.000,61","7.030,39","0,43%","-0,45%"
9,M2,"242.107.272,49",34.600,"3,46%","-222,42",173.041,"3,46%","5.997,86","6.997,87","7.024,73","0,38%","-0,02%"


In [17]:
# Calcula os desvios da classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_3 = base.groupby('classe_3', as_index=False).sum(numeric_only=True)
analise_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_3.soma_z_score / analise_classe_3.quantidade_historica)
analise_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_3.soma_z_score / analise_classe_3.quantidade_historica)
analise_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_3['desvio_valor'] = (analise_classe_3.valor_medio / analise_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_3['desvio_proporcao'] = (analise_classe_3.proporcao / analise_classe_3.proporcao_historica) - 1
analise_classe_3.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_3.proporcao_historica * analise_classe_3.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if np.isclose(meta_ponderada, float(meta), atol=1e-08) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_3.valor.sum()
soma_quantidade = analise_classe_3.quantidade.sum()
soma_proporcao = analise_classe_3.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_3.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_3.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_3=['Total'],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_3],
    ignore_index=True)

analise_classe_3.proporcao = analise_classe_3.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_3.proporcao_historica = analise_classe_3.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_3.desvio_valor = analise_classe_3.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_3.desvio_proporcao = analise_classe_3.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,"6.900.492.796,26",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","5.999,69","7.000,00","6.900,49","-1,42%","0,00%"
1,Z3,"293.636.508,14",59.809,"5,98%","-350.989,02",298.191,"5,96%","4.325,09","5.046,20","6.983,77","38,40%","0,29%"
2,Y3,"293.954.656,08",59.848,"5,98%","-351.356,98",298.381,"5,97%","4.324,40","5.045,39","6.882,06","36,40%","0,29%"
3,A3,"366.018.850,45",46.881,"4,69%","175.583,97",235.768,"4,72%","7.059,22","8.236,18","10.009,93","21,54%","-0,58%"
4,N3,"241.927.023,53",34.539,"3,45%","-24,25",173.066,"3,46%","5.999,49","6.999,77","7.060,94","0,87%","-0,21%"
5,R3,"241.243.667,43",34.475,"3,45%","-59,75",172.884,"3,46%","5.999,20","6.999,43","7.050,34","0,73%","-0,29%"
6,O3,"244.684.633,11",34.966,"3,50%","182,60",173.047,"3,46%","6.001,19","7.001,75","7.043,96","0,60%","1,03%"
7,J3,"243.218.446,47",34.750,"3,48%","191,67",172.838,"3,46%","6.001,27","7.001,84","7.033,58","0,45%","0,53%"
8,H3,"242.170.564,28",34.612,"3,46%","-11,49",173.438,"3,47%","5.999,60","6.999,89","7.030,39","0,44%","-0,22%"
9,M3,"241.373.422,22",34.487,"3,45%","79,12",173.042,"3,46%","6.000,34","7.000,76","7.024,73","0,34%","-0,35%"


In [18]:
# Calcula os desvios da classe_1 + classe_2 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1_classe_2 = base.groupby(['classe_1', 'classe_2'], as_index=False).sum(numeric_only=True)
analise_classe_1_classe_2['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade_historica)
analise_classe_1_classe_2['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1_classe_2.soma_z_score / analise_classe_1_classe_2.quantidade_historica)
analise_classe_1_classe_2['valor_medio'] = base.valor / base.quantidade
analise_classe_1_classe_2['desvio_valor'] = (analise_classe_1_classe_2.valor_medio / analise_classe_1_classe_2.meta_ajustada_com_z_score) - 1
analise_classe_1_classe_2['desvio_proporcao'] = (analise_classe_1_classe_2.proporcao / analise_classe_1_classe_2.proporcao_historica) - 1
analise_classe_1_classe_2.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_1_classe_2.proporcao_historica * analise_classe_1_classe_2.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if np.isclose(meta_ponderada, float(meta), atol=1e-08) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_1_classe_2.valor.sum()
soma_quantidade = analise_classe_1_classe_2.quantidade.sum()
soma_proporcao = analise_classe_1_classe_2.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1_classe_2.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1_classe_2.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_1_classe_2.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_1_classe_2 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_1_classe_2],
    ignore_index=True)

analise_classe_1_classe_2.proporcao = analise_classe_1_classe_2.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_2.proporcao_historica = analise_classe_1_classe_2.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_2.desvio_valor = analise_classe_1_classe_2.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_2.desvio_proporcao = analise_classe_1_classe_2.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1_classe_2.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,classe_2,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,"6.900.492.796,26",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","5.999,69","7.000,00","6.900,49","-1,42%","-0,00%"
1,Y1,Y2,"59.301.250,02",26.329,"2,63%","-351.477,51",131.535,"2,63%","2.198,07","2.564,54","6.991,54","172,62%","0,08%"
2,Z1,Z2,"59.738.480,59",26.390,"2,64%","-351.372,83",131.723,"2,63%","2.204,62","2.572,19","7.003,65","172,28%","0,17%"
3,I1,H2,"9.549.861,45",1.363,"0,14%","-60,48",6.678,"0,13%","5.986,81","6.984,97","7.216,22","3,31%","2,05%"
4,U1,P2,"9.257.728,58",1.323,"0,13%","7,40",6.616,"0,13%","6.001,28","7.001,86","7.202,52","2,87%","-0,02%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,C1,Y2,"9.375.274,72",1.344,"0,13%","7,91",6.782,"0,14%","6.001,35","7.001,94","6.783,67","-3,12%","-0,91%"
673,A1,A2,"134.463.577,33",13.815,"1,38%","175.519,04",69.098,"1,38%","9.613,56","11.216,39","10.009,93","-10,76%","-0,03%"
674,C1,C2,"134.223.233,98",13.814,"1,38%","175.703,78",69.226,"1,38%","9.610,67","11.213,02","7.068,32","-36,96%","-0,23%"
675,B1,B2,"134.694.286,92",13.859,"1,39%","175.822,49",69.190,"1,38%","9.614,99","11.218,06","7.034,43","-37,29%","0,15%"


In [19]:
# Calcula os desvios da classe_1 + classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_1_classe_3 = base.groupby(['classe_1', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_classe_1_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade_historica)
analise_classe_1_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_1_classe_3.soma_z_score / analise_classe_1_classe_3.quantidade_historica)
analise_classe_1_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_1_classe_3['desvio_valor'] = (analise_classe_1_classe_3.valor_medio / analise_classe_1_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_1_classe_3['desvio_proporcao'] = (analise_classe_1_classe_3.proporcao / analise_classe_1_classe_3.proporcao_historica) - 1
analise_classe_1_classe_3.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_1_classe_3.proporcao_historica * analise_classe_1_classe_3.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if np.isclose(meta_ponderada, float(meta), atol=1e-08) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_1_classe_3.valor.sum()
soma_quantidade = analise_classe_1_classe_3.quantidade.sum()
soma_proporcao = analise_classe_1_classe_3.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_1_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_1_classe_3.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_1_classe_3.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_1_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_1_classe_3],
    ignore_index=True)

analise_classe_1_classe_3.proporcao = analise_classe_1_classe_3.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_3.proporcao_historica = analise_classe_1_classe_3.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_3.desvio_valor = analise_classe_1_classe_3.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_1_classe_3.desvio_proporcao = analise_classe_1_classe_3.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_1_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,"6.900.492.796,26",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","5.999,69","7.000,00","6.900,49","-1,42%","0,00%"
1,Y1,Y3,"59.988.411,11",26.430,"2,64%","-351.501,82",131.581,"2,63%","2.199,13","2.565,79","6.991,54","172,49%","0,43%"
2,Z1,Z3,"59.517.153,78",26.361,"2,64%","-351.380,97",131.700,"2,63%","2.203,87","2.571,32","7.003,65","172,38%","0,08%"
3,I1,H3,"9.087.149,95",1.300,"0,13%","-0,03",6.577,"0,13%","5.999,69","6.999,99","7.216,22","3,09%","-1,17%"
4,U1,P3,"9.272.002,54",1.328,"0,13%","-41,87",6.630,"0,13%","5.990,71","6.989,52","7.202,52","3,05%","0,15%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,C1,Y3,"9.024.006,00",1.289,"0,13%","-31,46",6.652,"0,13%","5.992,96","6.992,15","6.783,67","-2,98%","-3,11%"
673,A1,A3,"134.661.743,17",13.838,"1,38%","175.461,51",69.276,"1,39%","9.603,09","11.204,18","10.009,93","-10,66%","-0,12%"
674,C1,C3,"134.221.315,05",13.819,"1,38%","175.649,12",69.229,"1,38%","9.609,39","11.211,53","7.068,32","-36,95%","-0,19%"
675,B1,B3,"134.651.423,81",13.852,"1,39%","175.814,36",69.186,"1,38%","9.615,03","11.218,11","7.034,43","-37,29%","0,11%"


In [20]:
# Calcula os desvios da classe_2 + classe_3 em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_classe_2_classe_3 = base.groupby(['classe_2', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_classe_2_classe_3['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade_historica)
analise_classe_2_classe_3['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_classe_2_classe_3.soma_z_score / analise_classe_2_classe_3.quantidade_historica)
analise_classe_2_classe_3['valor_medio'] = base.valor / base.quantidade
analise_classe_2_classe_3['desvio_valor'] = (analise_classe_2_classe_3.valor_medio / analise_classe_2_classe_3.meta_ajustada_com_z_score) - 1
analise_classe_2_classe_3['desvio_proporcao'] = (analise_classe_2_classe_3.proporcao / analise_classe_2_classe_3.proporcao_historica) - 1
analise_classe_2_classe_3.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_classe_2_classe_3.proporcao_historica * analise_classe_2_classe_3.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if np.isclose(meta_ponderada, float(meta), atol=1e-08) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_classe_2_classe_3.valor.sum()
soma_quantidade = analise_classe_2_classe_3.quantidade.sum()
soma_proporcao = analise_classe_2_classe_3.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_classe_2_classe_3.soma_z_score.sum()
soma_quantidade_historica = analise_classe_2_classe_3.quantidade_historica.sum()
soma_proporcao_historica = analise_classe_2_classe_3.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_classe_2_classe_3 = pd.concat([
    pd.DataFrame(dict(classe_2=['Total'],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_classe_2_classe_3],
    ignore_index=True)

analise_classe_2_classe_3.proporcao = analise_classe_2_classe_3.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2_classe_3.proporcao_historica = analise_classe_2_classe_3.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2_classe_3.desvio_valor = analise_classe_2_classe_3.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_classe_2_classe_3.desvio_proporcao = analise_classe_2_classe_3.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_classe_2_classe_3.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_2,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,"6.900.492.796,26",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","5.999,69","7.000,00","6.900,49","-1,42%","0,00%"
1,Z2,Z3,"59.210.470,10",26.314,"2,63%","-351.360,02",131.461,"2,63%","2.197,20","2.563,53","7.003,65","173,20%","0,08%"
2,Y2,Y3,"59.420.310,87",26.346,"2,63%","-351.474,69",131.662,"2,63%","2.201,76","2.568,86","6.991,54","172,17%","0,05%"
3,I2,H3,"9.504.622,28",1.360,"0,14%","19,52",6.788,"0,14%","6.003,78","7.004,77","7.216,22","3,02%","0,18%"
4,U2,P3,"9.099.775,32",1.298,"0,13%","-18,81",6.627,"0,13%","5.995,65","6.995,29","7.202,52","2,96%","-2,07%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,H2,K3,"8.914.850,38",1.271,"0,13%","61,67",6.604,"0,13%","6.012,98","7.015,50","6.796,66","-3,12%","-3,77%"
673,A2,A3,"134.586.137,18",13.829,"1,38%","175.525,57",69.164,"1,38%","9.610,24","11.212,52","10.009,93","-10,73%","-0,03%"
674,C2,C3,"133.988.112,46",13.780,"1,38%","175.658,04",69.127,"1,38%","9.614,90","11.217,96","7.068,32","-36,99%","-0,33%"
675,B2,B3,"135.018.534,05",13.902,"1,39%","175.881,45",69.128,"1,38%","9.619,45","11.223,26","7.034,43","-37,32%","0,55%"


In [21]:
# Calcula os desvios de todas as classes em relação à meta, utilizando a definição da meta ajustada
# A meta ajustada é calculada como: a meta mais a variação da meta vezes (a soma histórica do Z-score dividido pela soma da quantidade histórica de elementos)
analise_geral = base.groupby(['classe_1', 'classe_2', 'classe_3'], as_index=False).sum(numeric_only=True)
analise_geral['valor_medio_historico'] = media_base_historica + desvio_padrao_base_historica * (analise_geral.soma_z_score / analise_geral.quantidade_historica)
analise_geral['meta_ajustada_com_z_score'] = meta + variacao_meta * (analise_geral.soma_z_score / analise_geral.quantidade_historica)
analise_geral['valor_medio'] = base.valor / base.quantidade
analise_geral['desvio_valor'] = (analise_geral.valor_medio / analise_geral.meta_ajustada_com_z_score) - 1
analise_geral['desvio_proporcao'] = (analise_geral.proporcao / analise_geral.proporcao_historica) - 1
analise_geral.sort_values(by='desvio_valor', ascending=False, ignore_index=True, inplace=True)

meta_ponderada = (analise_geral.proporcao_historica * analise_geral.meta_ajustada_com_z_score).sum()
resultado = 'ela é igual à meta' if np.isclose(meta_ponderada, float(meta), atol=1e-08) else \
            'ela é diferente da meta'

print(f'A soma da meta ponderada com o Z-score é {meta_ponderada:_.2f} e {resultado}'.replace('.', ',').replace('_', '.') + '.')

soma_valor = analise_geral.valor.sum()
soma_quantidade = analise_geral.quantidade.sum()
soma_proporcao = analise_geral.proporcao.sum()
valor_medio = soma_valor / soma_quantidade

soma_z_score = analise_geral.soma_z_score.sum()
soma_quantidade_historica = analise_geral.quantidade_historica.sum()
soma_proporcao_historica = analise_geral.proporcao_historica.sum()
valor_medio_historico = media_base_historica + (desvio_padrao_base_historica * (soma_z_score / soma_quantidade_historica))

meta_ajustada_com_z_score = meta + variacao_meta * (soma_z_score / soma_quantidade)
desvio_valor = (valor_medio / meta_ajustada_com_z_score) - 1
desvio_proporcao = (soma_proporcao / soma_proporcao_historica) - 1

analise_geral = pd.concat([
    pd.DataFrame(dict(classe_1=['Total'],
                      classe_2=[''],
                      classe_3=[''],
                      valor=[soma_valor],
                      quantidade=[soma_quantidade],
                      proporcao=[soma_proporcao],
                      soma_z_score=[soma_z_score],
                      quantidade_historica=[soma_quantidade_historica],
                      proporcao_historica=[soma_proporcao_historica],
                      valor_medio_historico=[valor_medio_historico],
                      meta_ajustada_com_z_score=[meta_ajustada_com_z_score],
                      valor_medio=[valor_medio],
                      desvio_valor=[desvio_valor],
                      desvio_proporcao=[desvio_proporcao])),
    analise_geral],
    ignore_index=True)

analise_geral.proporcao = analise_geral.proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_geral.proporcao_historica = analise_geral.proporcao_historica.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_geral.desvio_valor = analise_geral.desvio_valor.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
analise_geral.desvio_proporcao = analise_geral.desvio_proporcao.apply(lambda x: f'{x:_.2%}'.replace('.', ',').replace('_', '.'))
display(analise_geral.map(lambda x: f'{x:_.2f}'.replace('.', ',').replace('_', '.') if type(x) == float else 
                                       f'{x:_.0f}'.replace('.', ',').replace('_', '.') if type(x) == int else x))

A soma da meta ponderada com o Z-score é 7.000,00 e ela é igual à meta.


,classe_1,classe_2,classe_3,valor,quantidade,proporcao,soma_z_score,quantidade_historica,proporcao_historica,valor_medio_historico,meta_ajustada_com_z_score,valor_medio,desvio_valor,desvio_proporcao
0,Total,,,"6.900.492.796,26",1.000.000,"100,00%","-0,00",5.000.000,"100,00%","5.999,69","7.000,00","6.900,49","-1,42%","0,00%"
1,M1,I2,B3,"385.519,33",53,"0,01%","-6,46",277,"0,01%","5.966,51","6.961,28","7.273,95","4,49%","-4,33%"
2,K1,A2,J3,"266.809,83",37,"0,00%","-14,53",261,"0,01%","5.920,46","6.907,56","7.211,08","4,39%","-29,12%"
3,B1,V2,T3,"421.099,86",58,"0,01%","-5,27",253,"0,01%","5.970,07","6.965,44","7.260,34","4,23%","14,62%"
4,A1,S2,I3,"409.820,02",57,"0,01%","-12,91",239,"0,00%","5.922,85","6.910,34","7.189,82","4,04%","19,25%"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17572,Y1,Y2,Y3,"50.420.509,78",25.060,"2,51%","-351.451,24",125.248,"2,50%","2.007,54","2.342,25","2.011,99","-14,10%","0,04%"
17573,Z1,Z2,Z3,"50.420.015,22",25.058,"2,51%","-351.387,74",125.243,"2,50%","2.008,10","2.342,90","2.012,13","-14,12%","0,04%"
17574,B1,B2,B3,"125.490.266,37",12.543,"1,25%","175.824,53",62.759,"1,26%","9.985,50","11.650,35","10.004,80","-14,12%","-0,07%"
17575,C1,C2,C3,"125.365.301,58",12.549,"1,25%","175.688,58",62.786,"1,26%","9.980,71","11.644,76","9.990,06","-14,21%","-0,07%"


# Fim